# LSTM Model for PV Power Forecasting

This notebook implements an LSTM (Long Short-Term Memory) neural network for forecasting photovoltaic power output.

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
import os

## Data Loading and Preprocessing

In [ ]:
# Load the processed training data
data_path = '../data/processed_training_data.parquet'
df = pd.read_parquet(data_path)

# Display the first few rows
print(f"Loaded data shape: {df.shape}")
df.head()

In [ ]:
# Feature selection
# Select relevant features for the model
features = ['GHI', 'DHI', 'DNI', 'Temperature', 'WindSpeed', 'CloudCover']
target = 'PowerOutput'

# Check for missing values
print("Missing values in dataset:")
print(df[features + [target]].isna().sum())

In [ ]:
# Data normalization
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

# Fit and transform the features
X = scaler_x.fit_transform(df[features])
# Reshape target to 2D array for scaling
y = scaler_y.fit_transform(df[[target]])

# Save the scalers for later use
joblib.dump(scaler_x, '../models/scaler_x.pkl')
joblib.dump(scaler_y, '../models/scaler_y.pkl')

## Time Series Data Preparation

In [ ]:
def create_sequences(X, y, time_steps=24):
    """
    Create sequences of data for time series forecasting.
    
    Args:
        X: Features array
        y: Target array
        time_steps: Number of time steps to look back
        
    Returns:
        X_seq: Sequences of features
        y_seq: Corresponding target values
    """
    X_seq, y_seq = [], []
    
    for i in range(len(X) - time_steps):
        X_seq.append(X[i:i + time_steps])
        y_seq.append(y[i + time_steps])
        
    return np.array(X_seq), np.array(y_seq)

In [ ]:
# Create sequences for LSTM
time_steps = 24  # Look back 24 time steps (e.g., hours)
X_seq, y_seq = create_sequences(X, y, time_steps)

print(f"Sequence shapes - X: {X_seq.shape}, y: {y_seq.shape}")

In [ ]:
# Split data into training and testing sets
train_size = int(len(X_seq) * 0.8)
X_train, X_test = X_seq[:train_size], X_seq[train_size:]
y_train, y_test = y_seq[:train_size], y_seq[train_size:]

print(f"Training set shapes - X: {X_train.shape}, y: {y_train.shape}")
print(f"Testing set shapes - X: {X_test.shape}, y: {y_test.shape}")

## LSTM Model Definition

In [ ]:
def build_lstm_model(input_shape, units=50, dropout_rate=0.2):
    """
    Build an LSTM model for time series forecasting.
    
    Args:
        input_shape: Shape of input data (time_steps, features)
        units: Number of LSTM units
        dropout_rate: Dropout rate for regularization
        
    Returns:
        model: Compiled LSTM model
    """
    model = Sequential()
    
    # First LSTM layer with return sequences for stacking
    model.add(LSTM(units=units, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    
    # Second LSTM layer
    model.add(LSTM(units=units))
    model.add(Dropout(dropout_rate))
    
    # Output layer
    model.add(Dense(1))
    
    # Compile the model
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    
    return model

In [ ]:
# Build the LSTM model
input_shape = (X_train.shape[1], X_train.shape[2])  # (time_steps, features)
model = build_lstm_model(input_shape)

# Display model summary
model.summary()

## Hyperparameter Tuning

In [ ]:
# Define hyperparameters to tune
hyperparameters = {
    'lstm_units': [32, 50, 64, 128],
    'dropout_rate': [0.1, 0.2, 0.3],
    'batch_size': [16, 32, 64],
    'learning_rate': [0.001, 0.01]
}

# Function to build and evaluate model with specific hyperparameters
def evaluate_model(lstm_units, dropout_rate, batch_size, learning_rate):
    # Build model with specified hyperparameters
    model = Sequential()
    model.add(LSTM(units=lstm_units, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units=lstm_units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    
    # Compile with specified learning rate
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    
    # Train with early stopping
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=5, restore_best_weights=True
    )
    
    history = model.fit(
        X_train, y_train,
        epochs=20,  # Reduced epochs for faster tuning
        batch_size=batch_size,
        validation_split=0.2,
        callbacks=[early_stopping],
        verbose=0
    )
    
    # Evaluate on validation data
    val_loss = min(history.history['val_loss'])
    return val_loss, model

In [ ]:
# Simple grid search implementation
# Note: For a more comprehensive approach, consider using libraries like Keras Tuner or scikit-learn's GridSearchCV

results = []

# Uncomment to run the grid search (warning: may take significant time)
'''
for units in hyperparameters['lstm_units']:
    for dropout in hyperparameters['dropout_rate']:
        for batch in hyperparameters['batch_size']:
            for lr in hyperparameters['learning_rate']:
                print(f"Testing: units={units}, dropout={dropout}, batch_size={batch}, lr={lr}")
                val_loss, _ = evaluate_model(units, dropout, batch, lr)
                results.append({
                    'lstm_units': units,
                    'dropout_rate': dropout,
                    'batch_size': batch,
                    'learning_rate': lr,
                    'val_loss': val_loss
                })
                
# Convert results to DataFrame and sort by validation loss
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('val_loss')
results_df.head(10)  # Show top 10 configurations
'''

# For demonstration, we'll use the following hyperparameters
best_params = {
    'lstm_units': 64,
    'dropout_rate': 0.2,
    'batch_size': 32,
    'learning_rate': 0.001
}

print("Using hyperparameters:")
for param, value in best_params.items():
    print(f"  {param}: {value}")

## Model Training with Best Hyperparameters

In [ ]:
# Define callbacks for early stopping and model checkpointing
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='../models/best_model.keras',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

In [ ]:
# Train the model
batch_size = 32
epochs = 50

history = model.fit(
    X_train, y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.2,
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)

## Model Evaluation

In [ ]:
# Plot training history
plt.figure(figsize=(12, 5))

# Plot loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot MAE
plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Model MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()

plt.tight_layout()
plt.savefig('../results/lstm_training_history.png')
plt.show()

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Inverse transform the predictions and actual values
y_test_inv = scaler_y.inverse_transform(y_test)
y_pred_inv = scaler_y.inverse_transform(y_pred)

# Calculate performance metrics
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
r2 = r2_score(y_test_inv, y_pred_inv)

print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R² Score: {r2:.4f}")

In [ ]:
# Plot predictions vs actual values
plt.figure(figsize=(14, 7))

# Plot a subset of the test data for better visualization
sample_size = min(500, len(y_test_inv))
indices = np.arange(sample_size)

plt.plot(indices, y_test_inv[:sample_size], 'b-', label='Actual Power Output')
plt.plot(indices, y_pred_inv[:sample_size], 'r-', label='Predicted Power Output')
plt.title('LSTM Model: Actual vs Predicted PV Power Output')
plt.xlabel('Time Steps')
plt.ylabel('Power Output (kW)')
plt.legend()
plt.grid(True)

plt.savefig('../results/lstm_predictions.png')
plt.show()

In [ ]:
# Create a scatter plot of actual vs predicted values
plt.figure(figsize=(10, 8))
plt.scatter(y_test_inv, y_pred_inv, alpha=0.5)
plt.plot([y_test_inv.min(), y_test_inv.max()], [y_test_inv.min(), y_test_inv.max()], 'r--')
plt.title('LSTM Model: Actual vs Predicted Power Output')
plt.xlabel('Actual Power Output (kW)')
plt.ylabel('Predicted Power Output (kW)')
plt.grid(True)

plt.savefig('../results/lstm_scatter.png')
plt.show()

## Save the Final Model

In [ ]:
# Save the model
model.save('../models/power_forecast_model.keras')
print("Model saved successfully.")

## Conclusion

This notebook demonstrates the implementation of an LSTM model for PV power forecasting. The model uses historical weather data and power output to predict future power generation. The performance metrics and visualizations provide insights into the model's accuracy and limitations.

Next steps could include:
1. Hyperparameter tuning to improve model performance
2. Feature engineering to incorporate additional relevant variables
3. Testing different model architectures (e.g., bidirectional LSTM, GRU)
4. Implementing ensemble methods for more robust predictions